## Runnable Interface
* The Runnable interface is the foundation for working with LangChain components, and it's implemented across many of them, such as language models, output parsers, retrievers, compiled LangGraph graphs and more.

The Runnable way defines a standard interface that allows a Runnable component to be:
* `Invoked`: A single input is transformed into an output.
* `Batched`: Multiple inputs are efficiently transformed into outputs.
* `Streamed`: Outputs are streamed as they are produced.
* `Inspected`: Schematic information about Runnable's input, output, and configuration can be accessed.
* `Composed`: Multiple Runnables can be composed to work together using the LangChain Expression Language (LCEL) to create complex pipelines.

Review the [LCEL Cheatsheet](https://python.langchain.com/docs/how_to/lcel_cheatsheet/) for some common patterns that involve the Runnable interface and LCEL expressions.

**Optimized parallel execution (batch):**

* LangChain Runnables offer a built-in `batch` (and `batch_as_completed`) API that allows you to process multiple inputs in parallel.
* Using these methods can significantly improve performance when needing to process multiple independent inputs, as the processing can be done in parallel instead of sequentially.

The two batching options are:
* `batch`: Process multiple inputs in parallel, returning results in the same order as the inputs.
* `batch_as_completed`: Process multiple inputs in parallel, returning results as they complete. Results may arrive out of order, but each includes the input index for matching.

**Streaming APIs:**

* Streaming is critical in making applications based on LLMs feel responsive to end-users. Runnables expose the following 3 streaming APIs:

1. sync stream and async astream: yields the output a Runnable as it is generated.

2. The async `astream_events`: a more advanced streaming API that allows streaming intermediate steps and final output.

3. The legacy async `astream_log`: a legacy streaming API that streams intermediate steps and final output.

Refer the [Streaming Conceptual Guide](https://python.langchain.com/docs/concepts/streaming/) for more details on how to stream in LangChain.

**Input and Output types:**

* Every Runnable is characterized by an input and output type. These input and output types can be any Python object, and are defined by the Runnable itself.
* Runnable methods that result in the execution of the Runnable (e.g., `invoke`, `batch`, `stream`, `astream_events`) work with these input and output types.
    * invoke: Accepts an input and returns an output.
    * batch: Accepts a list of inputs and returns a list of outputs.
    * stream: Accepts an input and returns a generator that yields outputs.

The input type and output type vary by component:


Component | Input Type | Output Type
----------|------------|---------------
Prompt | dictionary | PromptValue
ChatModel | a string, list of chat messages or a PromptValue | ChatMessage
LLM | a string, list of chat messages or a PromptValue | String
OutputParser | the output of an LLM or ChatModel | Depends on the parser
Retriever | a string | List of Documents
Tool | a string or dictionary, depending on the tool | Depends on the tool 

For more information about `Runnable`, refer: https://python.langchain.com/docs/concepts/runnables/

## Simple Chain

* It performs several actions in a particular order.
* Chains are sequences of actions with input or output data.
* Since the emergence of LCEL, LangChain is favouring LCEL chains over Traditional (Legacy) built-in Chains, but these are still maintained and frequently used.

In [1]:
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model = "gpt-3.5-turbo-0125")

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {politician}.")

chain = prompt | chatModel | StrOutputParser()

**Output Parser** automatically extracts the response content and return it. Otherwise the response comes in the form of AIMessage(content="....."). **StrOutputParser** returns the response content in the form of string.

In [4]:
chain.invoke({"politician": "Barack Obama"})

'Barack Obama is known for being left-handed, making him the sixth U.S. president to be left-handed.'

## LCEL

* LCEL has become the backbone of the newest versions of LangChain.
* Traditional chains (https://python.langchain.com/v0.1/docs/modules/chains/) are still supported, but treated as "Legacy" and have less functionality than the new LCEL chains.

**Main goals of LCEL**:

* Make it easy to build chains in a compact way.
* Support advanced LangChain functionality.

## Legacy Chain Example

In [5]:
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {soccer_player}.")

traditional_chain = LLMChain(
    llm=chatModel,
    prompt=prompt
)

traditional_chain.predict(soccer_player="Maradona")

<ipython-input-5-d4c6792a9a50>:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  traditional_chain = LLMChain(


'Diego Maradona, one of the greatest football players of all time, has a species of butterfly named after him. The "Diaethria maradonae" butterfly was discovered in the rainforests of Peru in 1999 and was named in honor of Maradona\'s famous "Hand of God" goal in the 1986 FIFA World Cup.'

In [6]:
# performing above operation using LCEL
chain = prompt | chatModel | StrOutputParser()
chain.invoke({"soccer_player": "Maradona"})

"One curious fact about Diego Maradona is that he has a species of slug named after him. The newly discovered species of slug, known as the 'Bieler's Maradona slug' was named in honor of the football legend due to its agility and speed, similar to Maradona's playing style on the field."

In [7]:
# another example
prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {soccer_player}.")

output_parser = StrOutputParser()

chain = prompt | chatModel | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Ronaldo is that he is known for his incredible work ethic and dedication to training. He reportedly spends hours in the gym and on the training field, constantly striving to improve his skills and maintain his peak physical condition. This level of commitment has played a significant role in his success as one of the greatest footballers of all time.'

## Runnable execution order in a LCEL chain

User's Input -> Prompt Template -> LLM Model -> Output Parser -> Final Output

In the above case,

User's Input = "soccer_player": "Ronaldo"

Prompt Template = "Tell me a curious fact about Ronaldo"

The output of LLM Model (in this case it is "chatModel") = AIMessage(content="One curious fact about Ronaldo is that he is known for ...")

The output of Output Parser = 'One curious fact about Ronaldo is that he is known for ...' , which is the Final Output.

In [ ]:
# Runnable execution alternatives i.e., stream, batch etc.
for s in chain.stream({"soccer_player": "Ronaldo"}):
    print(s, end="", flush=True)

Ronaldo is known for his incredible work ethic and dedication to his training regimen. He reportedly has a personal chef who travels with him to ensure he maintains a strict diet and nutrition plan, and he also reportedly sleeps in a hyperbaric chamber to aid in muscle recovery and overall performance.

In [9]:
chain.batch([{"soccer_player": "Ronaldo"}, {"soccer_player": "Messi"}])

['Ronaldo is known for his incredible work ethic and dedication to his fitness, but one curious fact about him is that he reportedly sleeps for an average of 5 naps per day, each lasting around 90 minutes. This unique sleep schedule is said to help him maintain his high energy levels and performance on the field.',
 'Lionel Messi holds the record for the most goals scored in a calendar year, with 91 goals in 2012.']